In [ ]:




    #list of particles for the decay channels, start and end masses

    #xw=0.231
    leptons=["11","13","15"]
    nu_alpha=["12","14","16"]
    '''    pid_start="111"
    pid_end="331"
    #self=model
    start_mass=self.masses(pid_start)
    end_mass=self.masses(pid_end)
    end_mass=10'''


    quarks_u_symbols=["u","c","t"]
    quarks_u=        ["2","4"]          #skipped top since it is heavy
    quarks_d_symbols=["d","s","b"]   
    quarks_d=        ["1","3","5"]


    charged_pseudo_symb=["K+" ,"pi+","D+" ,"D+s","B+" ,"Bc+"]
    charged_pseudo=     ["321","211","411","431","521","541"]


    neutral_pseudo_symb=["K0" ,"\eta","\eta'","pi0","D0" ,"B0" ,"B0s"]
    neutral_pseudo=     ["311","221" ,"331"  ,"111","421","511","531"]


    charged_vec_symb=["K*+","rho+","D*+","D*s+","B*+"]
    charged_vec=     ["323","213" ,"413","433" ,"523"]


    neutral_vec_symb=["K*0","\omega","\rho0","phi","J/psi","D*0"]#,"B*0","B*0s"]
    neutral_vec=     ["313","223"   ,"113"  ,"333","443"  ,"423"]#,"513","533" ]

    separate_channels=["1","2","3","4","5","113","213","223","333","443","221","331"]



    #functions to calculate decay widths for HNL
    #source for decay widths: https://arxiv.org/pdf/1005.1607.pdf

    #auxiliary functions 
    lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
    I1_integrand=lambda s,x,y,z: (12/s)*(s-x**2-y**2)*(1+z**2-s)*np.sqrt(lamda(s,x**2,y**2))*np.sqrt(lamda(1,s,z**2))
    I2_integrand=lambda s,x,y,z: (24*y*z/s)*(1+x**2-s)*np.sqrt(lamda(s,y**2,z**2))*np.sqrt(lamda(1,s,x**2))
    delta=lambda l1,l2: 1 if l1==l2 else 0
    xw=f"(0.231)" #xw=sin(theta_w)^2
    x=0
    xq=f"(self.masses(pid1)/mass)"
    y=xq
    z=xq

    #there is an extra coupling factor to allow all the actual couplings to get scaled if needed later
    coupling=1



    #for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
    def calc_br_la_lb_nub(self,pid1,pid2,pid3):
        yl1=f"(self.masses(pid1)/mass)"
        yl2=f"(self.masses(pid2)/mass)"
        ynul2=f"(self.masses(pid3)/mass)"
        x=yl1
        y=ynul2
        z=yl2
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        br_la_lb_nub="2*"+f"(hnl.vcoupling[str(abs(int(pid1)))]**2*coupling**2*hnl.GF()**2*mass**5/(192*np.pi**3))*{I1}*(1-delta(abs(int(pid1)),abs(int(pid2))))"
        return(br_la_lb_nub)

    #for N->l_beta^+ l_beta^- nu_alpha 
    def calc_br_lb_lb_nua(self,pid1,pid2,pid3):
        delta=lambda l1,l2: 1 if l1==l2 else 0
        if pid2=="11":
            l2=1
        if pid2=="13":
            l2=2
        if pid2=="15":
            l2=3
        if pid3=="12":
            l1=1
        if pid3=="14":
            l1=2
        if pid3=="16":
            l1=3
        xw=f"(0.231)"

        gL=f"(-(1/2)+{xw})"
        gR=f"{xw}"
        x=f"0"
        y=f"(self.masses(pid2)/mass)"
        z=y
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
        coupling1=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
        br_lb_lb_nua="2*"+f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**5/(192*np.pi**3))*(({gL}*{gR}+{delta(l1,l2)}*{gR})*{I2}+({gL}**2+{gR}**2+{delta(l1,l2)}*(1+2*{gL}))*{I1})"
        return(br_lb_lb_nua)

    #for N->U bar{D} l_alpha^-
    def calc_br_u_bd_l(self,pid1,pid2,pid3):
        xd=f"(self.masses(pid2)/mass)"
        xu=f"(self.masses(pid1)/mass)"
        xl=f"(self.masses(pid3)/mass)"
        x=xl
        y=xu
        z=xd
        I=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        coupling1=f"hnl.vcoupling[str(abs(int(pid3)))]"
        #this should be multiplied by 2 
        br_u_d_l="2*"+f"(hnl.VHHp(pid1,pid2)**2*hnl.GF()**2*mass**5*{coupling1}**2*coupling**2/(64*np.pi**3))*{I}"
        return(br_u_d_l)

    #HNL decays 3 body (quarks)
    #inclusive decays
    #for N-> q \bar{q} nu_l
    def calc_br_q_bq_nu(self,pid1,pid3):
        if pid1 in quarks_u:
            gL=f"(1/2-(2/3)*{xw})"
            gR=f"(-(2/3)*{xw})" #article had 2 g_R^U so I assumed one was supposed to have a D
        if pid1 in quarks_d:
            gL=f"(-1/2+(1/3)*{xw})"
            gR=f"((1/3)*{xw})"
        I1=f"quad(I1_integrand,({x}+{y})**2,(1-{z})**2,args=({x},{y},{z}))[0]"
        I2=f"quad(I2_integrand,({y}+{z})**2,(1-{x})**2,args=({x},{y},{z}))[0]"
        coupling1=f"hnl.vcoupling[str(abs(int(pid3))-1)]"
        #no factor of 2 since the charge conjugate process is the same process (I think the source already included factor of 2 here)
        #was a factor of 1/2 here
        br_q_bq_nu=f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**5/(32*np.pi**3))*({gL}*{gR}*{I2}+({gL}**2+{gR}**2)*{I1})"
        return(br_q_bq_nu)

    #for N->P+ l- 
    def calc_br_P_l(self,pid1,pid2):
        FP=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1+{x}**2)*(1+{x}**2-{y}**2)-4*{x}**2)"
        yP=f"(self.masses(pid1)/mass)"
        yl=f"(self.masses(pid2)/mass)"
        coupling1=f"hnl.vcoupling[str(abs(int(pid2)))]"
        #no factor of 2 added (probably should add)
        br_P_l="2*"+f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FP(yl,yP)}"
        return(br_P_l)

    def calc_br_P0_nu(self,pid1,pid2):
        coupling1=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
        #no factor of 2 added for conjugate process, one has already been added
        yP=f"(self.masses(pid1)/mass)"
        br_P0_nu=f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**3*(hnl.fH(pid1))**2/(64*np.pi))*(1-{yP}**2)**2"
        return(br_P0_nu)

    #for N->V+ l-
    def calc_br_V_l(self,pid1,pid2): 
        FV=lambda x,y: f"np.sqrt(lamda(1,{x}**2,{y}**2))*((1-{x}**2)**2+(1+{x}**2)*{y}**2-2*{y}**4)"
        #it was self.masses(pid1)
        yV=f"(self.masses(pid1)/mass)"
        yl=f"(self.masses(pid2)/mass)"
        coupling1=f"hnl.vcoupling[str(abs(int(pid2)))]"
        #no factor of 2 added, should probably add it though
        br_V_l="2*"+f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**3*hnl.fH(pid1)**2*hnl.VH(pid1)**2/(16*np.pi))*{FV(yl,yV)}"
        return(br_V_l)

    #for N->V0 nu
    def calc_br_V0_nu(self,pid1):
        xw=f"(0.231)"
        coupling1=f"hnl.vcoupling[str(abs(int(pid2))-1)]"
        yV=f"(self.masses(pid1)/mass)"
        if abs(int(pid1))==313 or abs(int(pid1)==333):
            kv=f"((-1/4)+{xw}/3)"
        if abs(int(pid1))==223 or abs(int(pid1))==113:
            kv=f"({xw}/3)"
        if abs(int(pid1))==423 or abs(int(pid1))==443:
            kv=f"((1/4)-2*{xw}/3)"
        #no need to multiply by 2 for conjugate process, since it is already accounted for
        br_V0_nu=f"({coupling1}**2*coupling**2*hnl.GF()**2*mass**3*hnl.fH(pid1)**2*{kv}**2/(2*np.pi))*(1-{yV}**2)**2*(1+2*{yV}**2)"
        return(br_V0_nu)


    #list of mesons which the HNL can decay to. 
    #only 3 body decays suggested by Jonathan


    def get_br_and_ctau(self,src_path):
        hnl.GF()=1.166378*10**(-5)

        pid_start="111"
        pid_end="331"
        #self=model
        start_mass=self.masses(pid_start)
        end_mass=self.masses(pid_end)
        end_mass=10

        #removes br files, since this part will create them again
        path=f"{src_path}/Models/HNL/model/br/*"
        path1=f"{src_path}/Models/HNL/model/br/"
        path1len=len(listdir(path1))
        if path1len!=0:
            remove_files_from_folder(path)
        remove_files_from_folder(path)

        #remove width files, so it can update it
        path=f"{src_path}/Models/HNL/model/width/*"
        path1=f"{src_path}/Models/HNL/model/width/"
        path1len=len(listdir(path1))
        if path1len!=0:
            remove_files_from_folder(path)

        #remove ctau file, so it can update it
        path=f"{src_path}/Models/HNL/model/ctau.txt"
        if exists(path)==True:
            os.remove(f"{src_path}/Models/HNL/model/ctau.txt")


        ##########creates width files############

        #for N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta
        pid1,pid2,pid3=["12","14","16"]
        mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
        #multiply by 2 to account for charge conjugate process
        br_nu_bnu_nu="2*"+f"(hnl.GF()**2*mass**5/(192*np.pi**3))*(hnl.vcoupling[str(abs(int(pid1))-1)]**2+hnl.vcoupling[str(abs(int(pid2))-1)]**2+hnl.vcoupling[str(abs(int(pid3))-1)]**2)"
        write_m_br_file(br_nu_bnu_nu,mass_produced,filename=f"model/width/width_3body_lep_sum_nua_bnub_nub.txt")
        print('3 body N->sum_{alpha beta} nu_alpha bar{nu}_beta nu_beta done...')

    '''
    #for N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta
    for a in range(len(leptons)):
        for b in range(len(leptons)):
            if a!=b:
                pid1=leptons[a]
                pid2=leptons[b]
                pid2="-"+pid2
                pid3=nu_alpha[b]
                mass_produced=(self.masses(pid1)+self.masses(pid2)+self.masses(pid3))
                br_la_lb_nub=calc_br_la_lb_nub(pid1,pid2,pid3)
                write_m_br_file(br_la_lb_nub,mass_produced,filename=f"model/width/width_3body_lep_la_lb_nub_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->l_{\alpha \neq \beta}^- l_beta^+ nu_beta done...')


    #for N->l_beta^+ l_beta^- nu_alpha using daniels formula
    for b in range(len(leptons)):
        for a in range(len(nu_alpha)):
            pid1=leptons[b]
            pid1="-"+pid1
            pid2=leptons[b]
            pid3=nu_alpha[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_lb_lb_nua=calc_br_lb_lb_nua(pid1,pid2,pid3)
            write_m_br_file(br_lb_lb_nua,mass_produced,filename=f"model/width/width_3body_lep_lb_lb_nua_{pid1}_{pid2}_{pid3}.txt", write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->l_beta^+ l_beta^- nu_alpha done...')


    #HNL decays 3 body (quarks)
    #inclusive decays
    #for N->U bar{D} l_alpha^-'
    for u in range(len(quarks_u)):
        for d in range(len(quarks_d)):
            for a in range(len(leptons)):
                pid1=quarks_u[u]
                pid2=quarks_d[d]
                pid3=leptons[a]
                mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
                br_u_d_l=calc_br_u_bd_l(pid1,pid2,pid3)
                write_m_br_file(br_u_d_l,mass_produced,filename=f"model/width/width_3body_inc_u_d_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N->U bar{D} l_alpha^- done...')


    #for N-> q \bar{q} nu_l
    for q in range(len(quarks_u+quarks_d)):
        pid1=(quarks_u+quarks_d)[q]
        pid2="-"+pid1
        for a in range(len(nu_alpha)):
            pid3=nu_alpha[a]
            mass_produced=self.masses(pid1)+self.masses(pid2)+self.masses(pid3)
            br_q_bq_nu=calc_br_q_bq_nu(pid1,pid3)
            write_m_br_file(br_q_bq_nu,mass_produced,filename=f"model/width/width_3body_inc_u_bu_l_{pid1}_{pid2}_{pid3}.txt",start_mass=start_mass,end_mass=10, write_pid=True,pid1_t=pid1,pid2_t=pid2, pid3_t=pid3)
    print('3 body N-> q \bar{q} nu_alpha done...')


    #pseudoscalar and vector decays
    #for N-> P+ l-
    for n in range(len(charged_pseudo)):
        pid1=charged_pseudo[n]
        for m in range(len(leptons)):
            if int(pid1)<0:
                pid2="-"+leptons[m]
            else:
                pid2=leptons[m]
            br_P_l=calc_br_P_l(pid1,pid2)
            mass_produced=self.masses(pid1)+self.masses(pid2)
            write_m_br_file(br_P_l,mass_produced,filename=f"model/width/width_2body_ex_P_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->P+ l- done...")


    #for N->P0 nu
    for n in range(len(neutral_pseudo)):
        pid1=neutral_pseudo[n]
        for m in range(len(nu_alpha)):
            if int(pid1)<0:
                pid2="-"+nu_alpha[m]
            else:
                pid2=nu_alpha[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            br_P0_nu=calc_br_P0_nu(pid1,pid2)
            write_m_br_file(br_P0_nu,mass_produced,filename=f"model/width/width_2body_ex_P0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->P0 nu done...")


    #for N->V+ l-
    for n in range(len(charged_vec)):
        pid1=charged_vec[n]
        for m in range(len(leptons)):
            if int(pid1)<0:
                pid2="-"+leptons[m]
            else:
                pid2=leptons[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            br_V_l=calc_br_V_l(pid1,pid2)
            write_m_br_file(br_V_l,mass_produced,filename=f"model/width/width_2body_ex_V_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)
    print("N->V+ l- done...")


    #for N->V0 nu
    for n in range(len(neutral_vec)):
        pid1=neutral_vec[n]
        br_V0_nu=calc_br_V0_nu(pid1)
        for m in range(len(nu_alpha)):
            if int(pid1)<0:
                pid2="-"+nu_alpha[m]
            else:
                pid2=nu_alpha[m]
            mass_produced=self.masses(pid1)+self.masses(pid2)
            write_m_br_file(br_V0_nu,mass_produced,filename=f"model/width/width_2body_ex_V0_{pid1}_{pid2}.txt",start_mass=start_mass,end_mass=end_mass, write_pid=True,pid1_t=pid1,pid2_t=pid2)


    print("N->V0 nu done...")
    write_m_ctau_file(filename="model/ctau.txt")
    print("ctau.txt created...")
    convert_width_to_br()
    print("br files created...")
    '''



#helpful functions for creating files and reading files, etc.
#takes a number and chops off part of it according to the precision, or delta
def chop(expr, delta=10**-2):
    return np.ma.masked_inside(expr, -delta, delta).filled(0)
#this portion will take a branching fraction and write to a file in the form m br
#for the decays of HNL, takes a branching ratio and creates a file of mass br
def write_m_br_file(br,mass_produced,filename="model/pi_nu.txt",start_mass=0,end_mass=10,write_pid=False,pid1_t=None,pid2_t=None,pid3_t=None):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    mass=0.01
    delm=0.01
    if write_pid==True:
        if pid3_t!=None:
            f.write(pid1_t + "\t" + pid2_t + "\t" + pid3_t + "\n")
        if pid3_t==None:
            f.write(pid1_t + "\t" + pid2_t + "\t" + "none" + "\n")
    if write_pid==False:
        f.write("None" + "\n")
    steps=int((10-0)/0.01)-1
    for n in range(steps):
        if mass>mass_produced and mass>start_mass and mass<end_mass:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))
        mass=chop(mass+delm)
    f.close()
    print(filename + ' created...')

#same as write_m_br_file except for mass between, start_mass, end_mass is for the starting mass for br_had and ending mass for br_had
def write_m_br_had_file(br,br_had,mass_produced,start_mass,end_mass,filename="model/pi_nu.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    mass=start_mass
    delm=.01
    steps=int((end_mass-start_mass)/.01)+1
    for n in range(steps):
        if mass>mass_produced:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br_had)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))
        mass=chop(mass+delm)
    mass=end_mass
    delm=.01
    steps=int((10-end_mass)/.01)+1
    for n in range(steps):
        if mass>mass_produced:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        else:
            f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,0]))

        mass=chop(mass+delm)
    f.close()
    print(filename + 'file created...')

#this is for the decay N->nu_l1 l2 l2
def write_m_brm_file(filename="model/pi_nu.txt"):
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    mass=.01
    br=bra_nul1_l2_l2(mass,l1,l2)+"*6.58*10**-14"
    delm=.01
    steps=int(10/.01)-1
    for n in range(steps):
        f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass,eval(br)]))
        mass=chop(mass+delm)
    f.close()
    print('file created...')

def calc_m_tau(filename="model/tau_GeV.txt"):
    br_files=os.listdir("model/width/")
    lep_file_char="width_3body_lep"
    had_file_char="width_3body_inc"
    counter=0
    for n in range(len(br_files)):
        if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char:
            counter+=1
            file_name=br_files[n]
            m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
            m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
            if counter==1:
                width=np.array(m_width_df_temp['width'].tolist())
            if counter!=1:
                width_temp=np.array(m_width_df_temp['width'].tolist())
                width+= width_temp
    mass=np.array(m_width_df_temp['mass'].tolist())
    tau=1/width
    return(mass,tau)

def calc_m_ctau(filename="model/ctau.txt"):
    br_files=os.listdir("model/width/")
    lep_file_char="width_3body_lep"
    had_file_char="width_2body_ex"
    hadf_file_char="width_3body_inc"
    counter=0
    for n in range(len(br_files)):
        if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char or br_files[n][0:len(hadf_file_char)]==hadf_file_char:
            counter+=1
            file_name=br_files[n]
            m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
            m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
            if counter==1:
                width=np.array(m_width_df_temp['width'].tolist())
            if counter!=1:
                width_temp=np.array(m_width_df_temp['width'].tolist())
                width+= width_temp
    mass=np.array(m_width_df_temp['mass'].tolist())
    ctau=((1/(width*10**-7))*((6.582122*pow(10.,-25.))))*3*10**8
    return(mass,ctau)

def write_m_ctau_file(filename="model/ctau.txt"):
    br_files=os.listdir("model/width/")
    lep_file_char="width_3body_lep"
    had_file_char="width_2body_ex"
    hadf_file_char="width_3body_inc"
    counter=0
    for n in range(len(br_files)):
        if br_files[n][0:len(lep_file_char)]==lep_file_char or br_files[n][0:len(had_file_char)]==had_file_char or br_files[n][0:len(hadf_file_char)]==hadf_file_char:
            counter+=1
            file_name=br_files[n]
            m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,skiprows=1)
            m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
            if counter==1:
                width=np.array(m_width_df_temp['width'].tolist())
            if counter!=1:
                width_temp=np.array(m_width_df_temp['width'].tolist())
                width+= width_temp
    mass=np.array(m_width_df_temp['mass'].tolist())
    ctau=((1/(width*10**-7))*((6.582122*pow(10.,-25.))))*3*10**8
    f = open(filename, "w+")
    f.close()
    f = open(filename, "a")
    if len(mass)!=len(ctau):
        print("the masses dont match the shape of ctau!!!")
    for n in range(len(mass)):
        f.write(" {: <10} {: <10}\n".format(*['%.2f' % mass[n],ctau[n]]))
    f.close()

def read_m_br(characteristic):
    br_files=os.listdir("model/width/")
    width=[]
    counter=0
    for n in range(len(br_files)):
        if br_files[n][0:len(characteristic)]==characteristic:
            counter+=1
            file_name=br_files[n]
            m_width_df_temp = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
            if counter==1:
                width=np.array(m_width_df_temp['width'].tolist())
            if counter!=1:
                width_temp=np.array(m_width_df_temp['width'].tolist())
                width+= width_temp
    mass=np.array(m_width_df_temp['mass'].tolist())
    return (mass,width,counter)

#"/Users/alechewitt/Desktop/felix_pts.csv"
def read_digitized_file(filename="/Users/alechewitt/Desktop/felix_pts.csv",name_col1="x",name_col2="y"):
    x_y_df = pd.read_csv(filename,delimiter=',',engine='python',names=[name_col1,name_col2],index_col=False)
    x=np.array(x_y_df[name_col1].tolist())[4:]
    y=np.array(x_y_df[name_col2].tolist())[4:]
    x=[float(a) for a in x]
    y=[float(a) for a in y]
    return(x,y)

def read_file(filename,name_col1="x",name_col2="y"):
    x_y_df = pd.read_csv(filename,delimiter='       ',engine='python',names=[name_col1,name_col2],index_col=False)
    x=np.array(x_y_df[name_col1].tolist())[4:]
    y=np.array(x_y_df[name_col2].tolist())[4:]
    return(x,y)

def convert_width_to_br():
    br_files=os.listdir("model/width/")
    ctau=pd.read_csv(f"model/ctau.txt",delimiter='       ',engine='python',names=['mass','ctau'],index_col=False)
    for n in range(len(br_files)):
        file_name=br_files[n]
        m_width_df = pd.read_csv(f"model/width/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,skiprows=1)
        m_width_df['width']=m_width_df['width']*ctau['ctau']*10**-7/(((6.582122*pow(10.,-25.)))*3*10**8)
        numpy_array = m_width_df.to_numpy()
        new_file_name="br"+br_files[n][len("width"):]
        np.savetxt(f"model/br/{new_file_name}", numpy_array,delimiter='       ')

def convert_width_to_br_temp(path="model/width_temp/"):
    br_files=os.listdir(path)
    ctau=pd.read_csv(f"model/ctau.txt",delimiter='       ',engine='python',names=['mass','ctau'],index_col=False)
    for n in range(len(br_files)):
        file_name=br_files[n]
        m_width_df = pd.read_csv(path + f"{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False)
        m_width_df['width']=m_width_df['width']*ctau['ctau']*10**-7/(((6.582122*pow(10.,-25.)))*3*10**8)
        numpy_array = m_width_df.to_numpy()
        new_file_name="br"+br_files[n][len("width"):]
        np.savetxt(f"model/br_temp/{new_file_name}", numpy_array,delimiter='       ')



#input path to given folder and it removes all files in that folder
def remove_files_from_folder(path):
    files = glob.glob(path)
    for f in files:
        os.remove(f)


def add_br_files(files,br_or_width):
    folder=f"{br_or_width}"
    br_files=os.listdir(f"model/{folder}/")
    counter=0
    for n in range(len(br_files)):
        for m in range(len(files)):
            if br_files[n][0:len(files[m])] in files:
                counter+=1
                file_name=br_files[n]
                m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
                m_width_df_temp['width'] = m_width_df_temp['width'].fillna(0)
                print("counter, should go up to 1 ",counter)
                if counter==1:
                    width=np.array(m_width_df_temp['width'].tolist())
                if counter!=1:
                    width_temp=np.array(m_width_df_temp['width'].tolist())
                    width+= width_temp
                    for l in range(len(width_temp)):
                        if width_temp[l]<0:
                            print(file_name)
                            pass
    mass=np.array(m_width_df_temp['mass'].tolist())
    return(mass,width)

#for group of mesons, takes a list of file names and adds the widths
def add_br_files_group(files,br_or_width):
    folder=f"{br_or_width}"
    counter=0
    for m in range(len(files)):
        file_name=files[m]+".txt"
        if counter==0:
            m_width_df = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
            counter+=1
            width=np.array(m_width_df['width'].tolist())
            mass=np.array(m_width_df['mass'].tolist())


        if counter!=0:
            m_width_df_temp = pd.read_csv(f"model/{folder}/{file_name}",delimiter='       ',engine='python',names=['mass','width'],index_col=False,dtype='float64',skiprows=1)
            width_temp=np.array(m_width_df_temp['width'].tolist())
            width+=width_temp
    return(mass,width)